### Necessary Imports & Declarations

In [1]:
from typing import Optional, TypeVar, Type

import instructor
from pydantic import BaseModel, Field
from openai import OpenAI, AsyncOpenAI
from langchain.schema.messages import BaseMessage
from langchain.schema.runnable import RunnableLambda
from langchain.adapters.openai import convert_message_to_dict

T = TypeVar('T', bound=BaseModel)

### Function to create an instructor integrated `runnable`

> Supports OpenAI compatible API. [Ollama, vLLM, ...]

In [2]:
def create_instructor_runnable(
    mode: instructor.Mode,
    model: str,
    response_model: Type[T],
    base_url: Optional[str] = None,
    api_key: Optional[str] = None,
    **kwargs
) -> RunnableLambda[str | list[BaseMessage], T]: 
    """
    Create an instructor integrated langchain runnable.
    @param mode: instructor mode to use.
    @param model: Model to use.
    @param response_model: pydantic class to format output.
    @param base_url: Base URL to get chat completions.
    @param api_key: API key to use for requesting base URL.
    @param kwargs: Extra kwargs to pass to create method.
    """
    
    client = instructor.patch(
        OpenAI(base_url=base_url, api_key=api_key), mode=mode)
    aclient = instructor.patch(
        AsyncOpenAI(base_url=base_url, api_key=api_key), mode=mode)
    
    load_kwargs = (
        lambda x: {
            'messages':([{'role': 'user', 'content': x}] 
                if isinstance(x, str) 
                else [convert_message_to_dict(msg) for msg in x]),
            'model': model,
            'response_model': response_model,
        } | kwargs 
    )

    def func(__arg):
        return client.chat.completions.create(**load_kwargs(__arg))
    
    async def afunc(__arg):
        return await aclient.chat.completions.create(**load_kwargs(__arg))

    return RunnableLambda(func=func, afunc=afunc)

### Usage

- Create a Response Model that represents how the final output should be.
- Create a instructor runnable 
- Invoke the runnable

In [3]:
class Character(BaseModel):
    name: str
    age: int
    fact: str = Field(..., description="A fact about the character")

In [4]:
llm = create_instructor_runnable(
    mode=instructor.Mode.JSON,
    model='llama2', 
    response_model=Character, 
    base_url="http://localhost:11434/v1", # Ollama default URL
    api_key="ollama" # not required, you can leave it empty
)

In [5]:
llm.invoke('Tell me about Elon Musk.')

Character(name='Elon Musk', age=49, fact='He is the CEO of SpaceX and Tesla, Inc.')

In [7]:
from langchain.schema.messages import SystemMessage, HumanMessage

await llm.ainvoke([
    SystemMessage(content='Answer query related to anime "Bleach"'),
    HumanMessage(content='Who is Ichigo Kurosaki.')])

Character(name='Ichigo Kurosaki', age=15, fact='He has the ability to turn into a Shinigami, a death god, and can use his Zanpakuto to fight against evil forces.')